In [24]:
import pandas as pd
import csv

In [22]:
# change file extensions
import os

def changeFileExtensions(folderPath, newFileType):
    folder = os.path.expanduser(folderPath)
    files = os.listdir(folder)

    for file in files:
        fileName, ext = os.path.splitext(file)
        src = os.path.join(folder, file)  
        dst = os.path.join(folder, fileName + newFileType) 
        os.rename(src, dst)


In [23]:
changeFileExtensions('/home/ymatsumu/cmse890_project/code/spectra', '.csv')